In [1]:
import numpy as np
import cv2 as cv
import time
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml # to download mnist data

In [2]:
def download(data_set = 'mnist', version='active', data_id=None):
    """
    Download given data_set

    Input: data_set, version, data_id
    data_set - name of the dataset in OpenML repository
    version - version of the dataset
    data_id - most exact way of distinguishing a dataset in OpenML repository

    Output: raw_data_x, raw_data_y
    raw_data_x - NxD
    """
    
    if data_id is not None:
        raw_data = fetch_openml(data_id = data_id)
        
    elif data_set == 'mnist':
        raw_data = fetch_openml('mnist_784')
        
    else:
        try:
            raw_data = fetch_openml(name = data_set, version = version)
        except:
            print('Dataset does not exist in OpenML repository.')

    # separate numeric data from classification target
    raw_data_x = np.array((raw_data['data']+0.5)/256.0)
    raw_data_y = np.array(raw_data['target'].astype('int8'))
    data_dim = raw_data_x.shape[1]
    
    return raw_data_x, raw_data_y, data_dim

In [3]:
raw_data_x, raw_data_y, data_dim = download('mnist')

In [4]:
print(raw_data_x.shape)
print(raw_data_y.shape)
print(raw_data_x)
print(raw_data_y)
print(data_dim)

(70000, 784)
(70000,)
[[0.00195312 0.00195312 0.00195312 ... 0.00195312 0.00195312 0.00195312]
 [0.00195312 0.00195312 0.00195312 ... 0.00195312 0.00195312 0.00195312]
 [0.00195312 0.00195312 0.00195312 ... 0.00195312 0.00195312 0.00195312]
 ...
 [0.00195312 0.00195312 0.00195312 ... 0.00195312 0.00195312 0.00195312]
 [0.00195312 0.00195312 0.00195312 ... 0.00195312 0.00195312 0.00195312]
 [0.00195312 0.00195312 0.00195312 ... 0.00195312 0.00195312 0.00195312]]
[5 0 4 ... 4 5 6]
784


In [5]:
def binarize(raw_data_x, raw_data_y):
    """
    From any classification dataset, pick out 0 and 1 labeled datapoints and create a new binary classification dataset with them.

    Input: raw_data_x, raw_data_y
    raw_data_x - NaxD matrix of numeric data. Each row is a datapoint.
    raw_data_y - Nax1 vector of corresponding class labels.

    Output: data_x, data_y, num_classes
    data_x - NbxD matrix of numeric data. Each row is a datapoint.
    data_y - Nbx1 vector of corresponding class labels; possible classes are 0 and 1.
    num_classes - 2
    """

    accptbl_pts = (raw_data_y == 0) | (raw_data_y == 1)
    data_x = raw_data_x[accptbl_pts]
    data_y = raw_data_y[accptbl_pts]
    num_classes = 2
    
    return data_x, data_y, num_classes

In [6]:
data_x, data_y, num_classes = binarize(raw_data_x, raw_data_y)

In [7]:
print(data_x.shape)
print(data_y.shape)
print(data_x)
print(data_y)
print(num_classes)

(14780, 784)
(14780,)
[[0.00195312 0.00195312 0.00195312 ... 0.00195312 0.00195312 0.00195312]
 [0.00195312 0.00195312 0.00195312 ... 0.00195312 0.00195312 0.00195312]
 [0.00195312 0.00195312 0.00195312 ... 0.00195312 0.00195312 0.00195312]
 ...
 [0.00195312 0.00195312 0.00195312 ... 0.00195312 0.00195312 0.00195312]
 [0.00195312 0.00195312 0.00195312 ... 0.00195312 0.00195312 0.00195312]
 [0.00195312 0.00195312 0.00195312 ... 0.00195312 0.00195312 0.00195312]]
[0 1 1 ... 1 0 1]
2


In [8]:
def split_data(x,y,fracs=[0.8,0.2],seed=0):
    """
    Randomly split data into two sets.

    Input: x, y, fracs=[0.8, 0.2], seed=0
    x - NxD matrix of x data (e.g. images)
    y - Nx1 vector of y data (e.g. labels)
    fracs - split fractions determining sizes of set one and set two.
    seed - random seed. 'None' disables the use of a new seed.

    Output: x1, y1, x2, y2
    x1 - (fracs[0]*N)xD matrix of x data of set 1
    y1 - (fracs[0]*N)x1 vector of y data of set 1
    x2 - (fracs[1]*N)xD matrix of x data of set 2
    y2 - (fracs[1]*N)x1 vector of y data of set 2
    """

    if seed is not None:
        np.random.seed(seed)
    N = x.shape[0]
    rp = np.random.permutation(N)

    N1 = int(fracs[0]*N)
    N2 = min(N-N1,int(fracs[1]*N))

    # split the data into two parts
    x1 = x[rp[:N1]]
    y1 = y[rp[:N1]]
    x2 = x[rp[N1:(N1+N2)]]
    y2 = y[rp[N1:(N1+N2)]]

    return x1,y1,x2,y2

In [9]:
portion = 0.01
fracs = [0.8,0.2]

train_x,train_y,test_x,test_y = split_data(data_x,data_y,fracs=[portion,portion], seed=None)
train_x,train_y,  val_x,val_y = split_data(train_x,train_y,fracs=fracs, seed=None)

num_train = len(train_y)
num_val = len(val_y)

In [10]:
print(train_x.shape)
print(train_y.shape)
print(num_train)
print(num_val)

(117, 784)
(117,)
117
29


In [11]:
def convert_2D_U8(data_x, img_sz): 
    """
    Grayscale images are sometimes represented as 1D vectors of [0,1] fractional entries.
    This function converts the representation to 2D matrices of [0,255] integer entries.

    Input: data_x, img_sz
    data_x - NxD matrix of all images in the dataset. Each row is one image.
    img_sz - (length, width) of the images in pixels (e.g. (28,28))

    Output: cnv_data_x
    cnv_data_x - (N x length x width) tensor of all images in the dataset with the new representation.
    """

    N = data_x.shape[0]
    length = img_sz[0]
    width = img_sz[1]

    cnv_data_x = data_x.reshape((N, length, width))
    cnv_data_x = (cnv_data_x*256).astype('uint8')

    return cnv_data_x

In [12]:
img_sz = (28,28)
cnv_train_x = convert_2D_U8(train_x, img_sz)
cnv_val_x = convert_2D_U8(val_x, img_sz)

In [13]:
print(cnv_train_x.shape)
print(cnv_val_x.shape)
print(cnv_train_x[0].shape)

(117, 28, 28)
(29, 28, 28)
(28, 28)


In [14]:
def sift(images, sift_params):
    """
    Extracts SIFT keypoints and descriptors for all input images.

    Input: images, sift_params
    images - (N x length x width) tensor of N grayscale images with size (length, width) and integer entries between [0,255]
    sift_params - parameters of the SIFT algorithm

    Output: kp, des
    kp - 
    des -
    """

    # define SIFT parameters. If no parameters are given, resort to default values.
    if sift_params is None:
        nfeatures = 0
        nOctaveLayers = 3
        contrastThreshold = 0.04
        edgeThreshold = 10
        sigma = 1.6
    else:
        nfeatures = sift_params['nfeatures']
        nOctaveLayers = sift_params['nOctaveLayers']
        contrastThreshold = sift_params['contrastThreshold']
        edgeThreshold = sift_params['edgeThreshold']
        sigma = sift_params['sigma']
        
    # create SIFT object with given parameters
    sift = cv.SIFT_create(nfeatures, nOctaveLayers, contrastThreshold, edgeThreshold, sigma)
    
    # extract SIFT keypoints and descriptors for all images
    N = images.shape[0]
    kp = []
    des = []
    for i in range(N):
        _kp, _des = sift.detectAndCompute(images[i],None)
        kp.append(_kp)
        des.append(_des)

    return kp, des

In [15]:
sift_params = {'nfeatures': 0, \
               'nOctaveLayers':3, \
               'contrastThreshold':0.04, \
               'edgeThreshold':10, \
               'sigma':1.6}
kp, des = sift(cnv_train_x, sift_params)

In [16]:
print(len(kp))
print(len(des))
print(des[0].shape)
print(kp[0])
print(des[0])

117
117
(10, 128)
(<KeyPoint 0x7fb8445c08a0>, <KeyPoint 0x7fb8445c0960>, <KeyPoint 0x7fb8445c0570>, <KeyPoint 0x7fb8445c0990>, <KeyPoint 0x7fb8445c04e0>, <KeyPoint 0x7fb8445c06c0>, <KeyPoint 0x7fb8445c0750>, <KeyPoint 0x7fb7d1845ea0>, <KeyPoint 0x7fb7d1845e70>, <KeyPoint 0x7fb7d1845f60>)
[[ 13.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  2.   0.   1. ...   0.   0.   1.]
 ...
 [ 11.  37.  66. ...   0.   0.   0.]
 [  6.   0.   0. ...  30.  56. 102.]
 [ 20.   0.   0. ...   1.   5.   1.]]


In [17]:
def kpnum_floor(kp, des, cutoff):
    """
    Receive SIFT keypoints and descriptors of a number of images. Discard all images with less than a certain number of keypoints.

    Input: kp, des, cutoff
    kp - list of tuples of keypoints of images.
    des - list of nx128 matrices of descriptors of corresponding images.
    cutoff - minimum acceptable number of keypoints in each image.

    Output: ct_kp, ct_des, ct_ind
    ct_kp - list of tuples of keypoints of images with more than the minimum number of keypoints.
    ct_des - list of nx128 matrices of descriptors of corresponding images.
    ct_ind - indices of images with more than the minimum number of keypoints in the original dataset.
    """

    N = len(kp)
    ct_ind = [i for i in range(N) if len(kp[i])>=cutoff]
    ct_kp = [kp[i] for i in ct_ind]
    ct_des = [des[i] for i in ct_ind]

    return ct_kp, ct_des, ct_ind

In [18]:
ct_kp, ct_des, ct_ind = kpnum_floor(kp, des, 2)
num_train = len(ct_ind)

In [19]:
print(len(ct_kp))
print(len(ct_des))
print(len(ct_ind))
print(ct_ind)
print(ct_kp[1])
print(ct_des[1].shape)

106
106
106
[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 116]
(<KeyPoint 0x7fb7d10d50f0>, <KeyPoint 0x7fb7d10d5330>, <KeyPoint 0x7fb7d10d51b0>, <KeyPoint 0x7fb7d10d51e0>, <KeyPoint 0x7fb7d10d5210>, <KeyPoint 0x7fb7d10d5090>, <KeyPoint 0x7fb7d10d5180>, <KeyPoint 0x7fb7d10d5240>, <KeyPoint 0x7fb7d10d5120>, <KeyPoint 0x7fb7d10d5150>)
(10, 128)


In [156]:
def kpnum_ceil(kp, des, cutoff):
    """
    Receive SIFT keypoints and descriptors of a number of images.
    For images with more than 'cutoff' number of keypoints, keep only 'cutoff' keypoints based on their proximity to the top-left corner.
    For example, if cutoff is 2, keep only 1 keypoint closest to and 1 keypoint farthest from the top-left corner.
    This strategy hopefully leads to the widest possible coverage of the image by the keypoints' descriptors.
    -----
    IMPORTANT Note: If you are using this function alongside kpnum_floor, you should use kpnum_floor before this function.

    Input: kp, des, cutoff
    kp - list of tuples of keypoints of images.
    des - list of nx128 matrices of descriptors of corresponding images.
    cutoff - maximum acceptable number of keypoints in each image.

    Output: ct_kp, ct_des
    ct_kp - list of tuples of keypoints of images, now having fewer equal keypoints to the given maximum.
    ct_des - list of (n*128)x1 vectors of concatenated descriptors of all keypoints of corresponding images.
    """

    assert cutoff > 0

    ct_kp = []
    ct_des = []
    N = len(kp)

    if cutoff == 2:
        for i in range(N):
            # calculate keypoints' distances from the top-left corner
            dists = [np.sum(np.array(key.pt)**2) for key in kp[i]]

            # choose keypoints closest and farthest from the top-left corner
            left_pt = np.argmin(dists)
            right_pt = np.argmax(dists)

            # create new lists with chosen keypoints and the concatenation of their descriptors
            kept_kps = (kp[i][left_pt], kp[i][right_pt])
            cat_des = np.append(des[i][left_pt], des[i][right_pt], axis=0)
            ct_kp.append(kept_kps)
            ct_des.append(cat_des)

    else:
        nr = int(cutoff/2) # number of keypoints farthest from the top-left corner
        nl = cutoff - nr   # number of keypoints closest to the top-left corner

        for i in range(N):
            # calculate keypoints' distances from the top-left corner
            n = len(kp[i])
            dists = [(j, np.sum(np.array(kp[i][j].pt)**2)) for j in range(n)]

            # sort keypoints by their distances
            sorted_dists = sorted(dists, key=lambda x:x[1])

            # keep the right number of keypoints closes to and farthest from the top-left corner
            raw_ind = [j if j < nl else -1*(j-nl+1) for j in range(cutoff)]
            kp_ind = [sorted_dists[ind][0] for ind in raw_ind]

            kept_kps = tuple([kp[i][ind] for ind in kp_ind])
            cat_des = des[i][kp_ind,:].reshape(-1)

            ct_kp.append(kept_kps)
            ct_des.append(cat_des)

    return ct_kp, ct_des

In [157]:
cct_kp, cct_des = kpnum_ceil(ct_kp, ct_des, 2)

In [166]:
print(len(cct_kp))
print(type(cct_des))
for i in range(len(cct_kp)):
    if len(cct_kp[i]) != 2 or cct_des[i].shape[0] != 256:
        print("Trouble!")

106
<class 'list'>


In [167]:
def normalize(descriptors):
    """
    Receive feature vectors describing a number of images (e.g. concatenated SIFT descriptors). Normalize the vectors according to below algorithm.
    There are two steps in this normalization: feature-wise and sample-wise (aka. image-wise).
        1. feature-wise: subtract mean and divide by standard deviation of each feature for all images.
        2. sample-wise: normalize l2-norm of each vector to 1.

    Input: descriptors
    descriptors - NxDf matrix of vectors describing N images.

    Output: nrm_dess
    nrm_dess - NxDf matrix of normalized vectors describing N images.
    """

    # feature-wise normalization
    means = np.mean(descriptors, axis=0, keepdims=True)
    stds = np.std(descriptors, axis=0, keepdims=True)
    nrm_dess = descriptors - means
    nrm_dess = nrm_dess / (stds + 0.01)

    # sample-wise normalization
    norms = np.linalg.norm(nrm_dess, axis=1, keepdims=True)
    nrm_dess = nrm_dess / (norms + 0.01)

    return nrm_dess

In [172]:
nrm_des = normalize(np.array(cct_des))

In [207]:
print(nrm_des.shape)
for i in range(nrm_des.shape[0]):
    if np.abs(np.linalg.norm(nrm_des[i]) - 1) > 0.01:
        print("Trouble in mean #{} : {}".format(i, np.mean(nrm_des[i])))
        
assert (np.abs(np.mean(nrm_des, axis=0, keepdims=True)) < 0.1).all()

(106, 256)


In [212]:
assert type(nrm_des) == np.ndarray or False